In [2]:
import os
import glob
from tqdm import tqdm
import pandas as pd 
import openslide
from histoplus.extract import extract
from histoplus.helpers.segmentor import CellViTSegmentor
from histoplus.helpers.tissue_detection import detect_tissue_on_wsi

import json
# import ijson
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image, ImageDraw, ImageFont
from typing import Dict, Tuple, List

import math
from collections import Counter
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator

/home/sjhong/miniconda3/envs/cellvit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
INFERENCE_IMAGE_SIZE = 588
MPP = 0.25

output_dir = './output/KBSMC_HEATMAP_test/'
os.makedirs(output_dir, exist_ok = True)

df = pd.read_csv("/home/sjhong/document/sampling_test.csv")
sample_list = glob.glob()
sample_list = [x for x in sample_list if x.split('/')[-1][:-4] in df['Patient_id'].tolist()]
print(len(sample_list))

66


In [17]:
priority = df.iloc[[0,36,37,65],1].tolist()
last = df.loc[~df.index.isin([0,36,37,65]), df.columns[1]].tolist()
sample_names = priority + last
sample_names

['1-007_05',
 '1-084_12',
 '1_122_17',
 '1-047_10',
 '1_160_19',
 '1_115_17',
 '1-054_10',
 '1_120_17',
 '1-049_10',
 '1-015_07',
 '1-046_10',
 '1-008_06',
 '1-066_11',
 '1_125_17',
 '1-035_09',
 '1-075_11',
 '1_147_19',
 '1-031_09',
 '1_148_19',
 '1-060_11',
 '1-048_10',
 '1-039_09',
 '1-021_08',
 '1-050_10',
 '1_121_17',
 '1-064_11',
 '1-011_06',
 '1_126_17',
 '1-063_11',
 '1-013_07',
 '1-033_09',
 '1-025_08',
 '1_152_19',
 '1_116_17',
 '1-040_09',
 '1-019_07',
 '1-043_10',
 '1-022_08',
 '1-006_04',
 '1-030_09',
 '1-036_09',
 '1_105_16',
 '1-059_11',
 '1-055_10',
 '1-002_04',
 '1-037_09',
 '1_134_18',
 '1-057_11',
 '1_128_18',
 '1-073_11',
 '1-058_11',
 '1-042_10',
 '1_153_19',
 '1_112_17',
 '1-069_11',
 '1-056_10',
 '1-080_11',
 '1-068_11',
 '1_132_18',
 '1-074_11',
 '1_129_18',
 '1_155_19',
 '1_106_16',
 '1-065_11',
 '1-020_07',
 '1-009_06']

In [18]:
sample_list = [f'/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/{x}.svs' for x in sample_names]
print(len(sample_list))

66


In [19]:
sample_list[0], sample_list[-1]

('/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-007_05.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-009_06.svs')

In [20]:
for  wsi_path  in tqdm(sample_list):
    
    if not os.path.exists(wsi_path):
        tqdm.write(wsi_path, "doesn't exsisted.")
        continue
    
    file_name = wsi_path.split('/')[-1][:-4]
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 64, # 786 에러
        ) 

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

  0%|          | 0/66 [00:00<?, ?it/s]2025-10-10 11:47:51.998 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 11:47:52.001 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 11:47:54.244 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 4161 tiles with Otsu's threshold.
2025-10-10 11:47:56.312 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  2%|▏         | 1/66 [06:45<7:19:38, 405.82s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-007_05_588.json


2025-10-10 11:54:37.195 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-10-10 11:54:37.196 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-10-10 11:54:40.126 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 5206 tiles with Otsu's threshold.
2025-10-10 11:54:41.752 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
  3%|▎         | 2/66 [14:06<7:34:31, 426.11s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-084_12_588.json


2025-10-10 12:01:57.790 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 12:01:57.792 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 12:02:02.748 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 9263 tiles with Otsu's threshold.
2025-10-10 12:02:05.966 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  5%|▍         | 3/66 [32:00<12:38:00, 721.91s/it]

...

OK → ./output/KBSMC_HEATMAP_test//seg_1_147_19_588.json


2025-10-10 15:11:36.997 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 15:11:36.999 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 15:11:40.330 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6864 tiles with Otsu's threshold.
2025-10-10 15:11:42.939 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 27%|██▋       | 18/66 [3:36:47<10:07:43, 759.66s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-031_09_588.json


2025-10-10 15:24:38.774 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.410944 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.262842.
2025-10-10 15:24:38.775 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.525684 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.262842.
 27%|██▋       | 18/66 [3:36:52<9:38:21, 722.94s/it] 


KeyboardInterrupt: 

['/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-080_11.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-068_11.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1_132_18.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-074_11.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1_129_18.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1_155_19.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1_106_16.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-065_11.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-020_07.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-009_06.svs']

In [22]:
for  wsi_path  in tqdm(sample_list[-8:]):
    
    if not os.path.exists(wsi_path):
        tqdm.write(wsi_path, "doesn't exsisted.")
        continue
    
    file_name = wsi_path.split('/')[-1][:-4]
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 64, # 786 에러
        ) 

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

  0%|          | 0/8 [00:00<?, ?it/s]2025-10-10 15:26:36.191 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 15:26:36.192 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 15:26:41.152 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 5222 tiles with Otsu's threshold.
2025-10-10 15:26:43.392 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 12%|█▎        | 1/8 [09:44<1:08:13, 584.75s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1_132_18_588.json


2025-10-10 15:36:20.694 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-10-10 15:36:20.696 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-10-10 15:36:24.239 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 3298 tiles with Otsu's threshold.
2025-10-10 15:36:26.379 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
 25%|██▌       | 2/8 [20:25<1:01:47, 617.89s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-074_11_588.json


2025-10-10 15:47:02.125 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 15:47:02.126 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
 25%|██▌       | 2/8 [20:30<1:01:30, 615.00s/it]


KeyboardInterrupt: 

In [23]:
for  wsi_path  in tqdm(list(reversed(sample_list[-26::2]))):
    
    if not os.path.exists(wsi_path):
        tqdm.write(wsi_path, "doesn't exsisted.")
        continue
    
    file_name = wsi_path.split('/')[-1][:-4]
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 64, # 786 에러
        ) 

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

  0%|          | 0/13 [00:00<?, ?it/s]2025-10-10 15:47:10.512 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 15:47:10.514 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 15:47:13.093 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 1586 tiles with Otsu's threshold.
2025-10-10 15:47:17.282 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  8%|▊         | 1/13 [05:24<1:04:53, 324.43s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-020_07_588.json


2025-10-10 15:52:35.288 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.410944 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.262842.
2025-10-10 15:52:35.289 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.525684 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.262842.
2025-10-10 15:52:40.845 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 17363 tiles with Otsu's threshold.
2025-10-10 15:52:43.095 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.262842 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.262842.
 15%|█▌        | 2/13 [37:50<3:54:23, 1278.46s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1_106_16_588.json


2025-10-10 16:25:01.652 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 16:25:01.659 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 16:25:08.028 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 15305 tiles with Otsu's threshold.
2025-10-10 16:25:10.563 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 23%|██▎       | 3/13 [59:08<3:33:02, 1278.23s/it]

...

OK → ./output/KBSMC_HEATMAP_test//seg_1-080_11_588.json


2025-10-10 17:13:34.516 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-10-10 17:13:34.517 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-10-10 17:13:37.552 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 7990 tiles with Otsu's threshold.
2025-10-10 17:13:39.978 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
 46%|████▌     | 6/13 [1:49:14<2:10:33, 1119.14s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-069_11_588.json


2025-10-10 17:36:25.868 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 17:36:25.869 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
 46%|████▌     | 6/13 [1:49:18<2:07:31, 1093.05s/it]


KeyboardInterrupt: 

In [24]:
recurr = ['1-048_10',	'1-039_09',	'1-050_10',	'1-011_06',	'1_152_19',	'1-022_08',	'1-084_12',	'1-069_11']
[x for x in sample_list if x.split('/')[-1][:-4] in recurr]

['/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-084_12.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-048_10.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-039_09.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-050_10.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-011_06.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1_152_19.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-022_08.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-069_11.svs']

In [25]:
for  wsi_path  in tqdm([x for x in sample_list if x.split('/')[-1][:-4] in recurr]):
    
    if not os.path.exists(wsi_path):
        tqdm.write(wsi_path, "doesn't exsisted.")
        continue
    
    file_name = wsi_path.split('/')[-1][:-4]
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 64, # 786 에러
        ) 

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

 12%|█▎        | 1/8 [00:00<00:01,  5.16it/s]2025-10-10 17:36:40.782 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 17:36:40.783 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 17:36:46.111 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6163 tiles with Otsu's threshold.
2025-10-10 17:36:49.234 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 25%|██▌       | 2/8 [12:27<43:57, 439.50s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-048_10_588.json


2025-10-10 17:49:07.598 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 17:49:07.599 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 17:49:13.092 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 8566 tiles with Otsu's threshold.
2025-10-10 17:49:16.399 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 38%|███▊      | 3/8 [25:58<50:46, 609.40s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-039_09_588.json


2025-10-10 18:02:38.913 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 18:02:38.914 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 18:02:43.965 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 5743 tiles with Otsu's threshold.
2025-10-10 18:02:46.631 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 50%|█████     | 4/8 [39:01<45:10, 677.65s/it]

...

OK → ./output/KBSMC_HEATMAP_test//seg_1-011_06_588.json


2025-10-10 18:20:02.075 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 18:20:02.077 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 18:20:08.627 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 9642 tiles with Otsu's threshold.
2025-10-10 18:20:11.476 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 75%|███████▌  | 6/8 [1:13:04<31:48, 954.07s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1_152_19_588.json


2025-10-10 18:49:44.264 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 18:49:44.265 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-10 18:49:47.409 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 4681 tiles with Otsu's threshold.
2025-10-10 18:49:50.454 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
100%|██████████| 8/8 [1:39:24<00:00, 745.51s/it] 

OK → ./output/KBSMC_HEATMAP_test//seg_1-022_08_588.json


In [ ]:
for  wsi_path  in tqdm(sample_list):
    
    if not os.path.exists(wsi_path):
        tqdm.write(wsi_path, "doesn't exsisted.")
        continue
    
    file_name = wsi_path.split('/')[-1][:-4]
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 64, # 786 에러
        )

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

  0%|          | 0/66 [00:00<?, ?it/s]2025-10-10 19:16:04.128 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-10 19:16:04.130 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
 11%|█         | 7/66 [00:03<00:28,  2.08it/s]

1-054_10 cannot reshape array of size 6545280 into shape (174,14,192,14)


2025-10-10 19:16:07.875 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.410944 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.262842.
2025-10-10 19:16:07.877 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.525684 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.262842.
2025-10-10 19:16:24.975 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6728 tiles with Otsu's threshold.
2025-10-10 19:16:28.973 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.262842 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.262842.
 29%|██▉       | 19/66 [23:43<1:05:56, 84.19s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1_148_19_588.json


2025-10-10 19:39:48.035 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-10-10 19:39:48.037 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-10-10 19:39:50.519 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 4719 tiles with Otsu's threshold.
2025-10-10 19:39:53.154 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
 30%|███       | 20/66 [48:27<2:28:11, 193.30s/it]

...

2025-10-11 03:36:04.527 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-10-11 03:36:04.528 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-10-11 03:36:07.904 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 3223 tiles with Otsu's threshold.
2025-10-11 03:36:10.217 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
 97%|█████████▋| 64/66 [8:26:23<11:24, 342.04s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-065_11_588.json


2025-10-11 03:42:28.069 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-11 03:42:28.071 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-11 03:42:30.752 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 4988 tiles with Otsu's threshold.
2025-10-11 03:42:33.397 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
100%|██████████| 66/66 [8:35:51<00:00, 468.97s/it]

OK → ./output/KBSMC_HEATMAP_test//seg_1-009_06_588.json


: 